In [ ]:
!pip install -U spacy

In [ ]:
!pip install spacy_transformers

In [3]:
import spacy_transformers
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [4]:
import json
cv_data = json.load(open('/content/train_data.json', 'r'))

In [ ]:
cv_data

In [7]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [8]:
!python -m spacy init fill-config /content/base_config.cfg /content/config.cfg

2023-06-22 15:59:28.582079: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
def get_spacy_doc(file, data):
  nlp =spacy.blank('en')
  db= DocBin()
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot =annot['entities']
    ents = []
    entity_indices = []
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity =True
          break
      if skip_entity == True:
        continue
      entity_indices = entity_indices +list(range(start,end))
      try:
        span = doc.char_span(start,end,label =label, alignment_mode = 'strict')
      except:
        print("error")
        continue
      if span is None:
        err_data =str([start,end]) + "    " +str(text) + "\n"
        file.write(err_data)

      else:
        ents.append(span)
    try:
      doc.ents =ents
      db.add(doc)
    except:
      pass
  return db

In [10]:
from sklearn.model_selection import train_test_split
train, test =train_test_split(cv_data, test_size = 0.3)

In [12]:
file =open('error.txt', 'w', encoding ='utf8')
db =get_spacy_doc(file, train)
db.to_disk('train_data.spacy')
db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')
file.close()

 37%|███▋      | 52/140 [00:00<00:00, 104.52it/s]

error


100%|██████████| 60/60 [00:00<00:00, 71.01it/s]


In [13]:
!python -m spacy train /content/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

2023-06-22 16:04:21.044662: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2023-06-22 16:04:31,328] [INFO] Set up nlp object from config
[2023-06-22 16:04:31,357] [INFO] Pipeline: ['transformer', 'ner']
[2023-06-22 16:04:31,364] [INFO] Created vocabulary
[2023-06-22 16:04:31,365] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT 

In [28]:
nlp = spacy.load('/content/output/model-best')

In [30]:
doc = nlp('''Rohan Chaudhary rohanchaudhary1526@gmail.com | +91-7888787497 | LinkedIn: rohan_chaudhary
B13 Beas Kund Hostel, IIT Mandi, Himachal Pradesh, India
EDUCATION

Indian Institute of Technology, Mandi Himachal Pradesh, India
Bachelor of Technology - Computer Science and Engineering, GPA: 9.25/10 Dec 2021 -Present
Coursework: Data Structures and Algorithms, Information and Databases Systems, Mathematical Foundation of Computer Science, Data Science -1,2,3 (Data mining, Data visualization and Machine Learning)

Govt. Model Senior Secondary School Chandigarh, India
CBSE Senior Secondary Examination (PCM), Percentage: 94.4% 2019-2021
WORK EXPERIENCE
Incubate Up Remote
Software Engineer Intern Jan 2023 – April 2023
Assisted in a project of designing a web application which allowed clients to book meetings and other sessions with the experts of the relevant field. Actively contributed to the organization by fixing the existing issues and by implementing many new functionalities in the web application.
Tech Stack: ReactJS, PHP, MySQL, HTML and CSS, REST API
PROJECTS
Flight Seat Allotment System Guide: Prof Varun Dutt

Brief Description: Developed a web application (FlyHigh) through which a person can allot flight seats to requesting passengers keeping one-fourth (25%) of the seats reserved for ladies and senior citizens (>60 years of age). This project was built as a part of our Applied Databases Practicum Course.
Tools/Skills: Frontend: HTML, CSS, JavaScript Backend: PHP, MySQL
Resume Parser Guide: Prof Aditya Nigam

Brief Description: Created a project which aims at building model which take resume as an input, infer details from it and then whether predict whether the resume is capable of being selected for a certain job position. In other words, it is an ML model built to shortlist resumes.
Tools/Skills: Python, Pandas, Scikit-learn, Natural Language Processing, Named-Entity Recognition
Landslide Monitoring System

Brief Description: Designed a landslide monitoring system which tells how likely is the occurrence of a landslide with the help of a dataset collected from various places around the Mandi district.
Tools/Skills: Python, NumPy, Pandas, Matplotlib, Scikit-learn
SKILLS AND TECHNOLOGIES

Programming Languages: C / C++, Python, HTML, JavaScript, Kotlin

IDEs and Programming Tools: Visual Studio Code, Sublime Text, Jupyter Notebook, Spyder

Other Tools: MySQL, Git, Arduino, LaTeX, MATLAB, PostgreSQL
SCHOLASTIC ACHIEVEMENTS AND EXTRA CURRICULAR ACTIVITIES

Secured All India Rank 2493 in JEE Advanced 2021 out of 2 lakh participating candidates.

Secured position in the top 0.5 percent students by securing All India Rank 3918 in JEE Mains 2021

among 1 million participating candidates.

Core team member of the Space Technology and Astronomy Club (STAC) of IIT Mandi.

Volunteer at GCS (Guidance and Counselling Service), IIT Mandi for the year 2022-2023.

Helped in collecting sponsorships by being a part in the Sponsorship Team of our annual sports fest, Rann-Neeti’22.''')
for ent in doc.ents:
  print(ent.text, "    ->   ", ent.label_)

Rohan Chaudhary     ->    Name
rohan_chaudhary
     ->    Email Address
Beas Kund Hostel, IIT Mandi, Himachal Pradesh, India     ->    College Name
Indian Institute of Technology, Mandi Himachal Pradesh     ->    College Name
Bachelor of Technology - Computer Science and Engineering, GPA     ->    Degree
Govt. Model Senior Secondary School Chandigarh, India
CBSE Senior Secondary Examination (PCM), Percentage: 94.4% 2019-2021
WORK EXPERIENCE
Incubate Up Remote
Software Engineer Intern Jan 2023 – April 2023
Assisted in a project of designing a web application which allowed clients to book meetings and other sessions with the experts of the relevant field. Actively contributed to the organization by fixing the existing issues and by implementing many new functionalities in the web application.
Tech Stack: ReactJS, PHP, MySQL, HTML and CSS, REST API
PROJECTS
Flight Seat Allotment System Guide: Prof Varun Dutt
     ->    College Name
Aditya Nigam     ->    Name


In [22]:
!pip install PyMuPDF

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 67.9 MB/s eta 0:00:00


In [23]:
import sys, fitz

In [24]:
fname ='/content/Rohan Chaudhary RESUME 22Jun.pdf'
doc = fitz.open(fname)

In [25]:
doc =[page for page in doc]


In [ ]:
text=""
for page in doc:
  for w in page.widgets():
    print(f"Widget has the type {w.field_type_string}")
    # Do whatever else you want to do with the widget

In [31]:
!zip -r /content/full.zip /content/

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2023.06.20/ (stored 0%)
  adding: content/.config/logs/2023.06.20/13.38.27.140182.log (deflated 58%)
  adding: content/.config/logs/2023.06.20/13.37.45.310511.log (deflated 57%)
  adding: content/.config/logs/2023.06.20/13.38.57.163862.log (deflated 56%)
  adding: content/.config/logs/2023.06.20/13.38.56.279827.log (deflated 57%)
  adding: content/.config/logs/2023.06.20/13.37.15.717456.log (deflated 91%)
  adding: content/.config/logs/2023.06.20/13.38.13.258365.log (deflated 86%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 23%)
  adding: content/.config/conf

In [32]:
!zip -r /content/full_output.zip /content/output/

  adding: content/output/ (stored 0%)
  adding: content/output/model-best/ (stored 0%)
  adding: content/output/model-best/tokenizer (deflated 81%)
  adding: content/output/model-best/ner/ (stored 0%)
  adding: content/output/model-best/ner/model (deflated 8%)
  adding: content/output/model-best/ner/moves (deflated 74%)
  adding: content/output/model-best/ner/cfg (deflated 33%)
  adding: content/output/model-best/meta.json (deflated 66%)
  adding: content/output/model-best/vocab/ (stored 0%)
  adding: content/output/model-best/vocab/vectors.cfg (stored 0%)
  adding: content/output/model-best/vocab/lookups.bin (stored 0%)
  adding: content/output/model-best/vocab/vectors (deflated 45%)
  adding: content/output/model-best/vocab/strings.json (deflated 76%)
  adding: content/output/model-best/vocab/key2row (stored 0%)
  adding: content/output/model-best/config.cfg (deflated 61%)
  adding: content/output/model-best/transformer/ (stored 0%)
  adding: content/output/model-best/transformer/mod